In [0]:
#Required libraries import
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
import os

In [0]:
#creating spark session
spark = SparkSession.builder.appName("imageRecog").getOrCreate()

In [0]:
#setting directory path
os.chdir("/dbfs/FileStore/tables")

In [0]:
#registering a udf to load images
def loaddf(path):
    df = ImageSchema.readImages(path)
    return df
spark.udf.register("load images in data frame", loaddf)

Out[4]: <function __main__.load images in data frame>

In [0]:
#reading images in Dataframe
img_df= loaddf("/FileStore/tables/imagesnew")

In [0]:
#creating a udf to add column for classification
def addcol():
    df_col = img_df.withColumn("label", lit(0))
    return df_col

In [0]:
df=addcol()

In [0]:
#registering a udf for data generator with image augmntation, we can call it later with model if we want
def genfun(path):
    datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    generator = datagen.flow_from_directory(path, target_size=(150, 150), batch_size=32, class_mode='binary')
    return generator
spark.udf.register("data generator with image augmentation", genfun)

Out[8]: <function __main__.data generator with image augmentation>

In [0]:
#Partitioning the data frame to smaller chukes to get different workors activated
df = df.repartition(200)

In [0]:
#train, test split
train, test = df.randomSplit([0.8, 0.2])

In [0]:
#training with pre-built model to generate embadding and differnt matrices
import sparkdl
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
import mlflow
import mlflow.spark
# model: InceptionV3
# extracting feature from images
featurizer = DeepImageFeaturizer(inputCol="image",
                                 outputCol="features",
                                 modelName="InceptionV3")
# used as a multi class classifier
lr = LogisticRegression(maxIter=5, regParam=0.03, 
                        elasticNetParam=0.5, labelCol="label")
# define a pipeline model
sparkdn = Pipeline(stages=[featurizer, lr])
#starting ML flow to capture model and environment details
with mlflow.start_run():
    spark_model = sparkdn.fit(train) # start fitting or training


In [0]:
 #looging model result in MLflow layer
  from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="weightedPrecision")
 test_metric = evaluator.evaluate(spark_model.transform(test))
 mlflow.log_metric('test_' + evaluator.getMetricName(), test_metric)

In [0]:
mlflow.log_artifacts('test_' + evaluator.getMetricName(), test_metric)

In [0]:
#evaluating the model with classifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# evaluate the model with test set
evaluator = MulticlassClassificationEvaluator() 
tx_test = spark_model.transform(test)
print('F1-Score ', evaluator.evaluate(tx_test, 
                                      {evaluator.metricName: 'f1'}))
print('Precision ', evaluator.evaluate(tx_test,
                                       {evaluator.metricName:                    'weightedPrecision'}))
print('Recall ', evaluator.evaluate(tx_test, 
                                    {evaluator.metricName: 'weightedRecall'}))
print('Accuracy ', evaluator.evaluate(tx_test, 
                                      {evaluator.metricName: 'accuracy'}))

('F1-Score ', 1.0)
('Precision ', 1.0)
('Recall ', 1.0)
('Accuracy ', 1.0)